tensorboard demo

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os.path

from keras import backend as K


# dimensions of our images.
img_width, img_height = 56, 56

f_model = './model'
model_filename = 'cnn_model.json'
weights_filename = 'cnn_model_weights.hdf5'

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 500
nb_validation_samples = 300
epochs = 50
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'],
             )
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

checkpointer = ModelCheckpoint(filepath="./saved/weights.hdf5", verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=32, write_graph=True, 
                           write_grads=True, write_images=True, embeddings_freq=0, 
                           embeddings_layer_names=None, embeddings_metadata=None, 
                           embeddings_data=None)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpointer, tensorboard] )


print('save the architecture of a model')
json_string = model.to_json()
open(os.path.join(f_model, model_filename), 'w').write(json_string)
print('save weights')
model.save_weights(os.path.join(f_model, weights_filename))

Using TensorFlow backend.


Found 222 images belonging to 2 classes.
Found 49 images belonging to 2 classes.
Epoch 1/50
14/15 [===========================>..] - ETA: 0s - loss: 0.6522 - acc: 0.7205
Epoch 00001: val_loss improved from inf to 0.60658, saving model to ./saved/weights.hdf5
15/15 [==============================] - 4s 294ms/step - loss: 0.6447 - acc: 0.7267 - val_loss: 0.6066 - val_acc: 0.6886
Epoch 2/50
14/15 [===========================>..] - ETA: 0s - loss: 0.5138 - acc: 0.7655
Epoch 00002: val_loss improved from 0.60658 to 0.57269, saving model to ./saved/weights.hdf5
15/15 [==============================] - 3s 172ms/step - loss: 0.5129 - acc: 0.7623 - val_loss: 0.5727 - val_acc: 0.9561
Epoch 3/50
14/15 [===========================>..] - ETA: 0s - loss: 0.4263 - acc: 0.7869
Epoch 00003: val_loss improved from 0.57269 to 0.41814, saving model to ./saved/weights.hdf5
15/15 [==============================] - 2s 161ms/step - loss: 0.4276 - acc: 0.7761 - val_loss: 0.4181 - val_acc: 0.9781
Epoch 4/50
14/